<a href="https://colab.research.google.com/github/abuzar01440/abuzar-portfolio/blob/main/gant_chart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gantt Chart using python ---> Plotly library




In [1]:
# lets create gantt chart

In [11]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.figure_factory as ff
import pandas as pd
from datetime import datetime

In [105]:
df = pd.read_csv("/content/ActivitiesStartEnd1.csv")
df.head()

,Activity,StartMonth,EndMonth,MilestoneID1,MilestoneRealisationMonth1,Payment,MilestoneRealisationMonth2
0,"Initaite Order Modules ""A""",25-Feb,25-Feb,1.0,25-Feb,150000.0,NaN
1,"Manufacture Modules ""A""",25-Feb,25-Apr,NaN,NaN,NaN,NaN
2,"Final Inspections & Testing ""A""",25-Apr,25-Apr,2.0,25-Apr,100000.0,NaN
3,"Ship & Deliver to OTML (BNE) ""A""",25-May,25-May,NaN,NaN,NaN,NaN
4,"Ship to Tabubil (OTML Responsible) ""A""",25-Jun,25-Jun,NaN,NaN,NaN,NaN


In [107]:
df

,Activity,StartMonth,EndMonth,MilestoneID1,MilestoneRealisationMonth1,Payment,MilestoneRealisationMonth2
0,"Initaite Order Modules ""A""",25-Feb,25-Feb,1.0,25-Feb,150000.0,NaN
1,"Manufacture Modules ""A""",25-Feb,25-Apr,NaN,NaN,NaN,NaN
2,"Final Inspections & Testing ""A""",25-Apr,25-Apr,2.0,25-Apr,100000.0,NaN
3,"Ship & Deliver to OTML (BNE) ""A""",25-May,25-May,NaN,NaN,NaN,NaN
4,"Ship to Tabubil (OTML Responsible) ""A""",25-Jun,25-Jun,NaN,NaN,NaN,NaN
5,"Install Modules ""A""",25-Jul,25-Jul,NaN,NaN,NaN,NaN
6,"Initiate Order Modules ""B""",25-Apr,25-Apr,3.0,25-Apr,150000.0,NaN
7,"Manufacture Modules ""B""",25-Apr,25-Jun,NaN,NaN,NaN,NaN
8,"Final Inspections & Testing ""B""",25-Jun,25-Jun,4.0,25-Jun,100000.0,NaN
9,"Ship & Deliver to OTML (BNE) ""B""",25-Jul,25-Jul,NaN,NaN,NaN,NaN


In [108]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Activity                    20 non-null     object 
 1   StartMonth                  20 non-null     object 
 2   EndMonth                    20 non-null     object 
 3   MilestoneID1                9 non-null      float64
 4   MilestoneRealisationMonth1  9 non-null      object 
 5   Payment                     9 non-null      float64
 6   MilestoneRealisationMonth2  0 non-null      float64
dtypes: float64(3), object(4)
memory usage: 1.2+ KB


In [104]:

# Custom function to convert date format
def convert_date(date_str):
    if pd.isna(date_str) or date_str == '':
        return None
    try:
        # Convert '25-Feb' to datetime
        year, month = str(date_str).split('-')
        return pd.to_datetime(f'20{year}-{month}-01', format='%Y-%b-%d')
    except Exception as e:
        print(f"Error converting date '{date_str}': {str(e)}")
        return None

# Convert dates
df['StartMonth'] = df['StartMonth'].apply(convert_date)
df['EndMonth'] = df['EndMonth'].apply(convert_date)
df['MilestoneRealisationMonth1'] = df['MilestoneRealisationMonth1'].apply(convert_date)

# Prepare data for Gantt chart
task_data = []

for idx, row in df.iterrows():
    if pd.isna(row['StartMonth']) or pd.isna(row['EndMonth']):
        continue

    # Add main activity bar
    task_data.append({
        'Task': str(row['Activity']),
        'Start': row['StartMonth'],
        'Finish': row['EndMonth'] + pd.DateOffset(days=28),  # Make bars visible for same month activities
        'Resource': 'Task',
        'Payment': row['Payment'] if pd.notna(row['Payment']) else None
    })

    # Add milestone with payment if MilestoneID1 exists
    if pd.notna(row['MilestoneID1']) and pd.notna(row['MilestoneRealisationMonth1']) and pd.notna(row['Payment']):
        task_data.append({
            'Task': str(row['Activity']),
            'Start': row['MilestoneRealisationMonth1'],
            'Finish': row['MilestoneRealisationMonth1'] + pd.DateOffset(days=1),
            'Resource': 'Milestone',
            'Payment': row['Payment']
        })

# Create color scheme
colors = {
    'Task': 'rgb(46, 137, 205)',      # Blue for tasks
    'Milestone': 'rgb(198, 47, 105)'  # Red for milestones
}

# Create the Gantt chart
fig = ff.create_gantt(task_data,
                     colors=colors,
                     index_col='Resource',
                     show_colorbar=True,
                     group_tasks=True,
                     showgrid_x=True,
                     showgrid_y=True)

fig.update_layout(
    title='Project Timeline',
    xaxis_title='Date',
    yaxis_title='Activity',
    height=800,  # Increase height to accommodate all activities
    xaxis=dict(
        tickformat='%b %Y',
        dtick='M1',  # Show every month
        tickangle=45,
        range=[pd.to_datetime('2025-02-01'), pd.to_datetime('2027-04-01')]
    ),
    showlegend=False
)


# Display the chart
fig.show()

# Print confirmation of data processing
print("\nData processing complete. Number of tasks plotted:", len(task_data))
print("Current Date and Time (UTC):", datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'))
print("Current User's Login: abuzar01440")


Data processing complete. Number of tasks plotted: 29
Current Date and Time (UTC): 2025-01-28 18:06:09
Current User's Login: abuzar01440


In [109]:
# Custom function to convert date format
def convert_date(date_str):
    if pd.isna(date_str) or date_str == '':
        return None
    try:
        # Convert '25-Feb' to datetime
        year, month = str(date_str).split('-')
        return pd.to_datetime(f'20{year}-{month}-01', format='%Y-%b-%d')
    except Exception as e:
        print(f"Error converting date '{date_str}': {str(e)}")
        return None

# Convert dates
df['StartMonth'] = df['StartMonth'].apply(convert_date)
df['EndMonth'] = df['EndMonth'].apply(convert_date)
df['MilestoneRealisationMonth1'] = df['MilestoneRealisationMonth1'].apply(convert_date)

# Prepare data for Gantt chart
task_data = []

for idx, row in df.iterrows():
    if pd.isna(row['StartMonth']) or pd.isna(row['EndMonth']):
        continue

    # Add main activity bar with payment as Resource if payment exists
    payment_text = f'${row["Payment"]:,.0f}' if pd.notna(row['Payment']) else 'No Payment'
    task_data.append({
        'Task': str(row['Activity']),
        'Start': row['StartMonth'],
        'Finish': row['EndMonth'] + pd.DateOffset(days=28),
        'Resource': payment_text
    })

# Create color scheme - different colors for different payment amounts
colors = {
    'No Payment': 'rgb(169, 169, 169)',      # Gray for no payment
    '$50,000': 'rgb(255, 127, 14)',          # Orange for 50k
    '$100,000': 'rgb(44, 160, 44)',          # Green for 100k
    '$150,000': 'rgb(31, 119, 180)'          # Blue for 150k
}

# Create the Gantt chart
fig = ff.create_gantt(task_data,
                     colors=colors,
                     index_col='Resource',
                     show_colorbar=True,
                     group_tasks=True,
                     showgrid_x=True,
                     showgrid_y=True)

# Update layout
fig.update_layout(
    title='Project Timeline',
    xaxis_title='Date',
    yaxis_title='Activity',
    height=800,
    xaxis=dict(
        tickformat='%b %Y',
        dtick='M1',
        tickangle=45,
        range=[pd.to_datetime('2025-02-01'), pd.to_datetime('2027-04-01')]
    ),
    showlegend=True,  # Show legend with payment amounts
    legend=dict(
        title="Payment Amount",
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.02
    )
)

# Display the chart
fig.show()

# Print confirmation of data processing
print("\nData processing complete. Number of tasks plotted:", len(task_data))



Data processing complete. Number of tasks plotted: 20
Current Date and Time (UTC): 2025-01-28 18:11:17
